In [1]:
from quchem.Hamiltonian_Generator_Functions import *

In [2]:
Molecule = 'LiH'#LiH'
geometry = None
num_shots = 10000
HF_occ_index = [0,1,2] #[0, 1,2] # for occupied_orbitals_index_list

In [5]:
### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis='sto-3g',
                     multiplicity=1,
                     geometry=geometry)  # normally None!

Hamilt.Get_Molecular_Hamiltonian()
SQ_CC_ops, THETA_params = Hamilt.Get_ia_and_ijab_terms(Coupled_cluser_param=True)
# print('UCC operations: ', SQ_CC_ops)

HF_transformations = Hamiltonian_Transforms(Hamilt.MolecularHamiltonian, SQ_CC_ops, Hamilt.molecule.n_qubits)

QubitHam = HF_transformations.Get_Qubit_Hamiltonian_JW()
#print('Qubit Hamiltonian: ', QubitHam)
QubitHam_PauliStr = HF_transformations.Convert_QubitMolecularHamiltonian_To_Pauliword_Str_list(QubitHam)

In [ ]:
### Ansatz ###
# from quchem.Ansatz_Generator_Functions import *
#
# UCCSD = UCCSD_Trotter(SQ_CC_ops, THETA_params)
# Second_Quant_CC_JW_OP_list = UCCSD.SingleTrotterStep()
# PauliWord_list = Convert_QubitOperator_To_Pauliword_Str_list(Second_Quant_CC_JW_OP_list)
# HF_UCCSD_ansatz = Ansatz_Circuit(PauliWord_list, Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
# # THETA_params = [random.uniform(0, 2 * np.pi) for _ in range(len(THETA_params))]
# ansatz_Q_cicuit = HF_UCCSD_ansatz.Get_Full_HF_UCCSD_QC(THETA_params)
# print(ansatz_Q_cicuit)

In [4]:
### Graph Colouring
from quchem.Graph import *

List_PauliWords, HamiltonainCofactors = zip(*QubitHam_PauliStr)

attribute_dictionary = {'Cofactors': HamiltonainCofactors}

List_of_nodes, node_attributes_dict = Get_list_of_nodes_and_attributes(List_PauliWords,
                                                                       attribute_dictionary=attribute_dictionary)

G = nx.Graph()
G = Build_Graph_Nodes(List_of_nodes, G, node_attributes_dict=node_attributes_dict, plot_graph=False)
G = Build_Graph_Edges_COMMUTING_QWC_AntiCommuting(G, List_of_nodes, 'C', plot_graph=False)

# comp_G = Get_Complemenary_Graph(G, node_attributes_dict=node_attributes_dict, plot_graph=True) # <- not currently used

single_G, multi_G = Get_subgraphs(G, node_attributes_dict=node_attributes_dict)
s_colour = Colour_list_of_Graph(single_G, attribute_dictionary=attribute_dictionary, plot_graph=False,
                                strategy='largest_first')
m_colour = Colour_list_of_Graph(multi_G, attribute_dictionary=attribute_dictionary, plot_graph=False,
                                strategy='largest_first')

anti_commuting_sets = Get_unique_graph_colours(s_colour + m_colour)
anti_commuting_sets

Building Graph Edges: 100%|##########| 631/631 [00:03<00:00, 201.81it/s]


{0: [{'I0 I1 I2 I3 I4 I5 I6 I7 I8 I9 I10 I11': {'Cofactors': (-3.9344419569678446+0j)}}],
 1: [{'I0 I1 I2 I3 I4 I5 Z6 Z7 I8 I9 I10 I11': {'Cofactors': (0.0782363777898524+0j)}}],
 2: [{'I0 I1 I2 I3 I4 I5 I6 I7 Z8 Z9 I10 I11': {'Cofactors': (0.07823637778985235+0j)}}],
 3: [{'I0 I1 I2 I3 I4 I5 Z6 I7 I8 I9 I10 I11': {'Cofactors': (-0.21675429325000514+0j)}},
  {'I0 I1 I2 I3 I4 I5 Y6 X7 X8 Y9 I10 I11': {'Cofactors': (0.004217284878422762+0j)}},
  {'Y0 Y1 I2 I3 I4 I5 X6 X7 I8 I9 I10 I11': {'Cofactors': (-0.002472706153881529+0j)}},
  {'Y0 Z1 Z2 Y3 I4 I5 X6 X7 I8 I9 I10 I11': {'Cofactors': (0.002077887498395713+0j)}},
  {'Y0 Z1 Z2 Z3 Z4 Y5 X6 X7 I8 I9 I10 I11': {'Cofactors': (0.0025623897800114825+0j)}},
  {'Y0 Z1 Z2 Z3 Z4 Z5 Y6 Y7 Z8 Z9 Z10 Y11': {'Cofactors': (0.0009084689616229731+0j)}}],
 4: [{'I0 I1 I2 I3 I4 I5 I6 Z7 I8 I9 I10 I11': {'Cofactors': (-0.21675429325000514+0j)}},
  {'I0 I1 I2 I3 I4 I5 Y6 Y7 X8 X9 I10 I11': {'Cofactors': (-0.004217284878422762+0j)}},
  {'Y0 X1 I2 I3 I4 I5 X6

# ALCU

$$R = e^{-i\alpha \mathbb{X} /2} = cos(\frac{\alpha}{2}) \mathbb{1} − isin(\frac{\alpha}{2}) \mathbb{X}$$

Note that:

$$\mathbb{X} = i \sum_{k=1}^{n-1} \beta_{k} P_{k} P_{n}  $$

where: $\sum_{k=1}^{n-1} (\beta_{k})^{2} = 1$

Subbing in:
$$R = cos(\frac{\alpha}{2}) \mathbb{1} − isin(\frac{\alpha}{2}) i \sum_{k=1}^{n-1} \beta_{k} P_{k} P_{n}$$
$$R = cos(\frac{\alpha}{2}) \mathbb{1} + sin(\frac{\alpha}{2})  \sum_{k=1}^{n-1} \beta_{k} P_{k} P_{n}$$
Combining terms:
$$R = cos(\frac{\alpha}{2}) \mathbb{1} +  \sum_{\Omega=1}^{n-1} \beta_{\Omega} P_{\Omega}$$

where:
- $ \sum_{k=1}^{n-1} sin(\frac{\alpha}{2}) \times \beta_{k} =  \sum_{\Omega=1}^{n-1} \beta_{\Omega}$
- $  \sum_{k=1}^{n-1} P_{k} P_{n} =  \sum_{\Omega=1}^{n-1} P_{\Omega}$

NOW by letting:

- $P_{\Omega=0} = \mathbb{1}$
- $cos(\frac{\alpha}{2}) = \beta_{\Omega=0}$

$$R =  \sum_{\Omega=0}^{n-1} \beta_{\Omega} P_{\Omega}$$

again note:
$$\sum_{\Omega=0}^{n-1} (\beta_{\Omega})^2 = cos^{2}(\frac{\alpha}{2}) + \sum_{\Omega=1}^{n-1} (\beta_{\Omega})^{2} = cos^{2}(\frac{\alpha}{2}) + \sum_{k=1}^{n-1} sin^{2}(\frac{\alpha}{2}) \times (\beta_{k})^{2} = cos^{2}(\frac{\alpha}{2}) + sin^{2}(\frac{\alpha}{2}) \sum_{k=1}^{n-1}  (\beta_{k})^{2} $$

$$\sum_{k=1}^{n-1} (\beta_{k})^{2} = 1$$

$$ cos^{2}(\frac{\alpha}{2}) + sin^{2}(\frac{\alpha}{2}) \sum_{k=1}^{n-1}  (\beta_{k})^{2} = cos^{2}(\frac{\alpha}{2}) + sin^{2}(\frac{\alpha}{2}) = 1$$

THEREFORE:

$$ \sum_{\Omega=0}^{n-1} (\beta_{\Omega})^2 = 1$$


THIS is just a linear combination of unitary matrices!!!!!

We want to perform our Hamiltonian (of self inverse terms):

$$R = H =   \sum_{\Omega=0}^{n-1} \beta_{\Omega} P_{\Omega}$$

Let us define $V$ as a controlled $P_{\Omega}$:

$$ V = \sum_{\Omega=0}^{n-1} | \Omega \rangle \langle \Omega | \; \otimes P_{\Omega} $$

Taking $V$ to be self-inverse... means one can add ancilla qubit to construct self-inverse $U$:
$$
U = 
\begin{bmatrix}
0 & A^{\dagger}VA \\
A^{\dagger}VA & 0 \\
\end{bmatrix}
=
\begin{bmatrix}
A^{\dagger} & 0 \\
0 & A^{\dagger} \\
\end{bmatrix}
\begin{bmatrix}
0 & V \\
V & 0 \\
\end{bmatrix}
\begin{bmatrix}
A & 0 \\
0 & A \\
\end{bmatrix}
$$

Next we need to prepare ancilla line:
$$ | A \rangle = A| 0 \rangle^{\otimes log_{2}(\Omega)} = \sum_{\Omega=0}^{n-1} \xi_{\Omega} | \Omega \rangle$$


From definition we have:

$$\frac{H}{\lambda}| \psi \rangle = \langle 0| A^{\dagger} U A | 0 \rangle | \psi \rangle = \sum_{\Omega=0}^{n-1} \xi_{\Omega} \xi_{\Omega}^{*} P_{\Omega} | \psi \rangle$$ 


THEREOFRE WE REQUIRE:

$$H = \lambda \sum_{\Omega=0}^{n-1} \xi_{\Omega} \xi_{\Omega}^{*} P_{\Omega} = \sum_{\Omega=0}^{n-1} \beta_{\Omega} P_{\Omega}$$ 

THEREOFRE for all $\Omega (\{=0,1,2,...,n-1\})$ WE REQUIRE:

$$\lambda \xi_{\Omega} \xi_{\Omega}^{*} = \beta_{\Omega}$$ 

Taking the absolute value and summing

$$  \lambda \sum_{\Omega=0}^{n-1} |\xi_{\Omega} \xi_{\Omega}^{*}| =  \sum_{\Omega=0}^{n-1} |\beta_{\Omega}| \leq \lambda $$

OVERALL: We need $\lambda \xi_{\Omega} \xi_{\Omega}^{*}$ terms in ancilla state $| A \rangle$ to represent $\beta_{\Omega}$

$$ | A \rangle = \sum_{\Omega=0}^{n-1} \xi_{\Omega} | \Omega \rangle = \sum_{\Omega=0}^{n-1} \sqrt{\frac{\beta_{\Omega}}{\lambda}} | \Omega \rangle$$

CHECKING ACTION:

$$\frac{H}{\lambda}| \psi \rangle = \langle 0| A^{\dagger} U A | 0 \rangle | \psi \rangle $$

$$\frac{H}{\lambda}| \psi \rangle = \sum_{\Omega=0}^{n-1} \xi_{\Omega} \xi_{\Omega}^{*} P_{\Omega} | \psi \rangle $$


as: $\xi_{\Omega} = \sqrt{\frac{\beta_{\Omega}}{\lambda}} $

$$\frac{H}{\lambda}| \psi \rangle = \sum_{\Omega=0}^{n-1} \frac{\beta_{\Omega}}{\lambda} P_{\Omega} | \psi \rangle $$

cancal $\lambda$:

$$ H | \psi \rangle = \sum_{\Omega=0}^{n-1} \beta_{\Omega} P_{\Omega} | \psi \rangle $$


# Qubitization

- A self-inverse operation $U$ is equivalent to a reflection
- If we have a second reflection $R$
- then we get a rotation (a product of reflections = rotation... same thing in Grover's alg.) Note the spectrum given by a PRODUCT OF REFLECTIONS is given by *(https://arxiv.org/pdf/quant-ph/0401053.pdf, THEOREM 1 pg 6-7)

The eigenvalues correspond to angle of rotation for each eigenstate.

Applying the theorm shows that:

$$W = RU$$

has eigenvalues:

$$e^{\pm i \: arccos(\frac{h}{\lambda})}$$

for eigenvalue $h$ of $H$.

One can show (see lemma **)

$$ \langle G| W^{n} | G \rangle = T_{n}\big(\frac{H}{\lambda}\big) $$

where $T_{n}(.)$ is the $n^{th}$ Chebyshev polynomial of the first kind... originating from the relation that:

$$ T_{n}\big(cos(\theta)\big) = cos(n \theta) $$

where here $\theta=\frac{h}{\lambda}$

applying $W^{n}$ gives a rotation by $n\theta$

Let:

$$R = e^{-i\alpha \mathbb{X} /2} = cos(\frac{\alpha}{2}) \mathbb{1} − isin(\frac{\alpha}{2}) \mathbb{X}= e^{-i Ht}$$

The Jacobi-Anger expansion:

$$e^{-i H t}=J_{0}(-\lambda t)+2 \sum_{n=0}^{\infty} i^{n} J_{n}(-\lambda t) T_{n}(H / \lambda)$$

where $J_{n}(.)$ is the $n^{th}$ Bessel function of the first kind

note we have used: $ cos(\pm n \: arccos\big(\frac{H}{\lambda}\big)) = T_{n}\big(\frac{H}{\lambda}\big) $

THEREFORE

the polynomial in $W$ can be used to approximate $e^{-i H t}$ !!!!!
[This polynomial is generated by quantum signal processing!]

summary of problem:

we have A_LCU of H:

$$H = \lambda \sum_{\Omega=0}^{n-1} \xi_{\Omega} \xi_{\Omega}^{*} P_{\Omega} = \sum_{\Omega=0}^{n-1} \beta_{\Omega} P_{\Omega}$$ 

we want to perform:

$$A = f(H)$$

where: $H = \sum_{j} e^{i \theta_{j}} | j \rangle \langle j |$

In [2]:
# U_0----*-------U_1----*-------U_2----*-------U_3----*------ 
#        |              |              |              |
# -----|---|----------|---|----------|---|----------|---|----    ...  
# -----| H |----------| H |----------| H |----------| H |----
# -----|___|----------|___|----------|___|----------|___|----

top line has:
$$
\begin{bmatrix}
g_{00}(\theta) & g_{01}(\theta) \\
g_{10}(\theta) & g_{11}(\theta) \\
\end{bmatrix}
$$

Therefore we need to choose each $U_{i}$ such that:
$$
\begin{bmatrix}
g_{00}(\theta) & g_{01}(\theta) \\
g_{10}(\theta) & g_{11}(\theta) \\
\end{bmatrix}
=
\begin{bmatrix}
f(e^{i \theta}) & * \\
* & * \\
\end{bmatrix}
$$